In [1]:
import pandas as pd
import pickle
import tensorflow as tf
import keras
import warnings
import joblib

warnings.filterwarnings('ignore')

In [2]:
train = pd.read_excel('../Data/train.xlsx')
train.dropna(inplace=True)
train['data_type'] = 'train'

validation = pd.read_excel('../Data/validation.xlsx')
validation.dropna(inplace=True)
validation['data_type'] = 'validation'

test = pd.read_excel('../Data/test.xlsx')
test.dropna(inplace=True)
test['data_type'] = 'test'

df = pd.concat([train, test, validation])

In [3]:
lr = pickle.load(open('../Modelos/V1/LR_V1.pkl', 'rb'))
nb = pickle.load(open('../Modelos/V1/NB_V1.pkl', 'rb'))
xg = pickle.load(open('../Modelos/V1/XGB_V1.pkl', 'rb'))
lstm = keras.saving.load_model('../Modelos/V1/LSTM_V1/')

2024-02-23 11:27:44.969515: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-02-23 11:27:44.969546: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-23 11:27:44.969552: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-23 11:27:44.969582: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-23 11:27:44.969596: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
cv = joblib.load('../Modelos/vectorizer.joblib')

In [5]:
df

,name,gender,data_type
0,NARYIS,female,train
1,NOEMI ISABEL,female,train
2,SOUFIANE,male,train
3,ASTOU,female,train
4,ESTELLE,female,train
...,...,...,...
9863,ALVARO PATRICIO,male,validation
9864,JOSE ALEJO,male,validation
9865,TEIMURAZ,male,validation
9866,VICTORIA BEATRIZ,female,validation


In [6]:
encoded = cv.transform(df['name']).toarray()

In [7]:
encoded.shape

(106545, 72)

In [8]:
def gender_prediction(name: str, model, vectorizer):
    name_encoded = vectorizer.transform([name]).toarray()
    prediction = model.predict(name_encoded)

    if prediction >= 0.5:
        return 'Male'

    else:
        return 'Female'

In [9]:
df['lr_gender'] = lr.predict(encoded)
df['xg_gender'] = xg.predict(encoded)
df['nb_gender'] = nb.predict(encoded)
df['lstm_gender'] = lstm.predict(encoded)

   9/3330 [..............................] - ETA: 20s  

2024-02-23 11:27:46.605207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


3330/3330 [==============================] - 16s 5ms/step


In [10]:
df['gender'] = df['gender'].replace({'female': 'Female', 'male': 'Male'})

df['lr_gender'] = df['lr_gender'].replace({0: 'Female', 1: 'Male'})
df['xg_gender'] = df['xg_gender'].replace({0: 'Female', 1: 'Male'})
df['nb_gender'] = df['nb_gender'].replace({0: 'Female', 1: 'Male'})

df['lstm_gender'] = (df['lstm_gender'] >= 0.5).astype('int64')
df['lstm_gender'] = df['lstm_gender'].replace({0: 'Female', 1: 'Male'})

In [11]:
df.sample(50)

,name,gender,data_type,lr_gender,xg_gender,nb_gender,lstm_gender
1674,LUNA,Female,validation,Male,Female,Female,Female
55890,RUTH,Female,test,Male,Female,Male,Female
18871,ANGELA ROCIO,Female,test,Female,Male,Male,Female
6455,LUZ ELENA,Female,test,Female,Female,Female,Female
20071,FLOR MARIA,Female,test,Male,Female,Female,Female
9627,ROSA GEMA,Female,validation,Male,Female,Male,Female
11294,SANDRA MARIE,Female,train,Female,Female,Female,Female
18589,DIANA PAOLA,Female,test,Female,Female,Female,Female
3885,FLAVIO,Male,validation,Male,Male,Male,Female
1001,GLORIA JOSEFA,Female,validation,Male,Female,Male,Female


In [13]:
for j in ['lr_gender',	'xg_gender',	'nb_gender',	'lstm_gender']:
    print(df[j].value_counts())
    print('==============')

lr_gender
Female    53871
Male      52674
Name: count, dtype: int64
xg_gender
Female    57459
Male      49086
Name: count, dtype: int64
nb_gender
Female    58929
Male      47616
Name: count, dtype: int64
lstm_gender
Female    64059
Male      42486
Name: count, dtype: int64


In [18]:
df[df['gender']!=df['lstm_gender']].sample(50)

,name,gender,data_type,lr_gender,xg_gender,nb_gender,lstm_gender
3715,FABIO 0,Male,test,Male,Male,Male,Female
36175,GURUTZE,Female,train,Male,Female,Male,Male
7595,RUBEN 0,Male,test,Male,Male,Male,Female
19959,MARIA CONSTANZA,Female,test,Female,Male,Female,Male
25195,YENNYFER 0,Female,test,Male,Male,Female,Male
6309,ROMAIN,Male,validation,Male,Female,Female,Female
10674,MARTIN 0,Male,test,Male,Male,Female,Female
2781,KIRAN,Male,validation,Female,Male,Female,Female
30589,DANIEL RAMON,Male,train,Male,Male,Female,Female
13991,PIEDAD LORENA,Female,test,Male,Male,Male,Male


In [16]:
df.shape

(106545, 7)